<a href="https://colab.research.google.com/github/5arSanti/federico-object-recognition-model/blob/main/Train_YOLO_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, yaml
import glob
import os, shutil, pathlib
from zipfile import ZipFile
from utils.create_data_yaml import create_data_yaml

# Introduction

This notebook uses [Ultralytics](https://docs.ultralytics.com/) to train YOLO11, YOLOv8, or YOLOv5 object detection models with a custom dataset.

**Verify NVIDIA GPU Availability**

In [ ]:
!nvidia-smi

**Option 1. Upload the Dataset**

Upload the `data.zip` (Export from Label studio) to the root folder.

## 2. Split images into train and validation folders
Next, we'll unzip `data.zip` and create some folders to hold the images.

In [ ]:
# Unzip images to a custom data folder
!unzip -q dataset/data.zip -d dataset/custom_data

Afterwards and taking into account that we use Ultralytics, it is needed particular folder structure to store training data for models. The root folder is named “data”. Inside, there are two main folders:

*   **Train**: These are the actual images used to train the model. In one epoch of training, every image in the train set is passed into the neural network. The training algorithm adjusts the network weights to fit the data in the images.

*   **Validation**: These images are used to check the model's performance at the end of each training epoch.

In [ ]:
!python utils/train_val_split.py --datapath="dataset/custom_data" --train_pct=0.9

# 4.&nbsp;Configure Training


There's one last step before we can run training: we need to create the Ultralytics training configuration YAML file. This file specifies the location of your train and validation data, and it also defines the model's classes.

It is necessary to ensure that the Labelmap is located in `custom_data/classes.txt`.

In [ ]:
path_to_classes_txt = 'dataset/custom_data/classes.txt'
path_to_data_yaml = 'data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat data.yaml

# 5.&nbsp;Train Model

## 5.1 Training Parameters

**Model architecture & size (`model`):**

There are several YOLO11 models sizes available to train, including `yolo11n.pt`, `yolo11s.pt`, `yolo11m.pt`, `yolo11l.pt`, and `yolo11xl.pt`. Larger models run slower but have higher accuracy, while smaller models run faster but have lower accuracy. 

Example: [check it out here to get a sense of their speed accuracy](https://youtu.be/_WKS4E9SmkA). 
By default, `yolo11n.pt` will be used as the starting point.

We can also train with YOLOv8 or YOLOv5 models by substituting `yolo11` for `yolov8` or `yolov5`.


**Number of epochs (`epochs`)**
In machine learning, one “epoch” is one single pass through the full training dataset. Setting the number of epochs dictates how long the model will train for. The best amount of epochs to use depends on the size of the dataset and the model architecture. 

In our case we will use 60 epochs. 
If your dataset has more than 200 images, a good starting point is 40 epochs.


**Resolution (`imgsz`)**
Resolution has a large impact on the speed and accuracy of the model: a lower resolution model will have higher speed but less accuracy. 
YOLO models are typically trained and inferenced at a 640x640 resolution. 

We can choose to lower the resolution to 480x480 for better model performance


model = Specifies the model to use
epochs = Specifies the number of training epochs
imgsz = Specifies the resolution of the images

In [ ]:
!yolo detect train data=data.yaml model=yolo11n.pt epochs=60 imgsz=640

The training algorithm will parse the images in the training and validation directories and then start training the model. 

At the end of each training epoch, the program runs the model on the validation dataset and reports the resulting mAP, precision, and recall. As training continues, the mAP should generally increase with each epoch. Training will end once it goes through the number of epochs specified.

The best trained model weights will be saved in `content/runs/detect/train/weights/best.pt`. Additional information about training is saved in the `content/runs/detect/train` folder, including a `results.png` file that shows how loss, precision, recall, and mAP progressed over each epoch.

# 6.&nbsp;Test Model
The commands below run the model on the images in the validation folder and then display the results for the first 5 images.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
from IPython.display import Image, display
for image_path in glob.glob(f'runs/detect/predict/*.jpg')[:5]:
  display(Image(filename=image_path, height=400))
  print('\n')


The model should draw a box around each object of interest in each image.

# 7.&nbsp;Deploy Model

## 7.1 Download YOLO Model

First, zip and download the trained model by running the code blocks below.

The code creates a folder named `my_model`, moves the model weights into it, and renames them from `best.pt` to `my_model.pt`, and also adds the training results for reference.

In [ ]:
def get_notebook_path():
    return str(pathlib.Path().resolve())

print(get_notebook_path())

In [ ]:
base_dir = get_notebook_path()
train_dir = os.path.join(base_dir, "runs", "detect", "train")
output_dir = os.path.join(base_dir, "my_model")

os.makedirs(output_dir, exist_ok=True)

shutil.copy(os.path.join(train_dir, "weights", "best.pt"), os.path.join(output_dir, "my_model.pt"))
shutil.copytree(train_dir, os.path.join(output_dir, "train"), dirs_exist_ok=True)


# 8. Using the model

In [25]:
!python utils/yolo_detect.py --model my_model/my_model.pt --source usb0 --resolution 1280x720

[ WARN:0@1.615] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1.616] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
Unable to read frames from the camera. This indicates the camera is disconnected or not working. Exiting program.
Average pipeline FPS: 0.00
